# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [1]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions 

from pyspark.sql import functions as F
from pyspark.sql import types as T
import pandas as pd
import numpy as np
import datetime

spark = SparkSession.builder.appName("Pyspark Quiz").getOrCreate()

In [2]:
# load data
path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

# Question 1

Which page did user id "" (empty string) NOT visit?

In [13]:
# TODO: write your code to answer question 1
user_log.filter(user_log.userId == "").select("page").dropDuplicates().show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [15]:
# TODO: use this space to explore the behavior of the user with an empty string
user_log.filter(user_log.userId == "").collect()

[Row(artist=None, auth='Logged Out', firstName=None, gender=None, itemInSession=0, lastName=None, length=None, level='free', location=None, method='PUT', page='Login', registration=None, sessionId=5598, song=None, status=307, ts=1513721196284, userAgent=None, userId=''),
 Row(artist=None, auth='Logged Out', firstName=None, gender=None, itemInSession=26, lastName=None, length=None, level='paid', location=None, method='GET', page='Home', registration=None, sessionId=428, song=None, status=200, ts=1513721274284, userAgent=None, userId=''),
 Row(artist=None, auth='Logged Out', firstName=None, gender=None, itemInSession=5, lastName=None, length=None, level='free', location=None, method='GET', page='Home', registration=None, sessionId=2941, song=None, status=200, ts=1513722009284, userAgent=None, userId=''),
 Row(artist=None, auth='Logged Out', firstName=None, gender=None, itemInSession=5, lastName=None, length=None, level='paid', location=None, method='GET', page='Home', registration=None, 

# Question 3

How many female users do we have in the data set?

In [27]:
# TODO: write your code to answer question 3
user_log.filter(user_log.gender == 'F').drop_duplicates(["userId"]).count()

462

# Question 4

How many songs were played from the most played artist?

In [57]:
# TODO: write your code to answer question 4
user_log.dropna(how= "any", subset="artist").groupBy("artist").count().sort(F.desc("count")).first()

Row(artist='Coldplay', count=83)

# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [11]:
from pyspark.sql import Window

In [27]:
# TODO: write your code to answer question 5
visit_home_func = F.udf(lambda page: 1 if page == "Home" else 0, T.IntegerType())
windowval = Window.partitionBy("userId") \
            .orderBy(F.desc("ts")) \
            .rangeBetween(Window.unboundedPreceding, 0)

sdf_visit = user_log.select(["sessionId", "ts", "userId", "page"]) \
        .filter((user_log.page.isin(["NextSong", "Home"]))) \
        .withColumn("visit_home", visit_home_func("page")) \
        .withColumn("period_visit_home", F.sum("visit_home").over(windowval))

sdf_visit.filter(sdf_visit.page == 'NextSong') \
            .groupBy("userId", "period_visit_home") \
            .agg(F.count("period_visit_home")) \
            .agg(F.avg("count(period_visit_home)")) \
            .show()


num_songs =  user_log.filter(user_log.page == 'NextSong') \
        .groupBy("userId", "sessionId") \
        .agg(F.count("sessionId")) \
        .agg(F.avg("count(sessionId)")) \
        .show()

+-----------------------------+
|avg(count(period_visit_home))|
+-----------------------------+
|            6.898347107438017|
+-----------------------------+

+---------------------+
|avg(count(sessionId))|
+---------------------+
|    6.830605564648118|
+---------------------+

